In [5]:
# Importera bibliotek
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Ställ in stil
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Bibliotek importerade")

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
# Ladda data från SQLite
conn = sqlite3.connect('matkassen.db')
df = pd.read_sql('SELECT * FROM matkassen_tvattad', conn)
conn.close()

# Konvertera datumkolumner
date_cols = ['pren_startdatum', 'pren_avslutsdatum', 'paus_från_filled', 'paus_till_filled', 'leveransdatum']
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print(f"✓ Data laddat från databas: {df.shape[0]} rader, {df.shape[1]} kolumner")
print(f"\nKolumner: {list(df.columns)}")
print(f"\nDatatyper:\n{df.dtypes}")

In [ ]:
# Visa första raderna
df.head()

## METOD 1: Simple Aggregation (Enkel summering)

Gruppera och aggregera data för att få grundläggande KPIer.

In [ ]:
# === KPI 1: Genomsnittligt veckapris per kasstyp ===
kpi1_pris = df.groupby('kasstyp')['veckapris'].agg([
    ('Genomsnittlig Pris (SEK)', 'mean'),
    ('Min Pris (SEK)', 'min'),
    ('Max Pris (SEK)', 'max'),
    ('Antal Prenumerationer', 'count')
]).round(2)

print("KPI 1: Pris-analys per kasstyp")
print(kpi1_pris)
print()

In [ ]:
# === KPI 2: Leveransstatus per kasstyp ===
kpi2_leverans = pd.crosstab(
    df['kasstyp'], 
    df['leveransstatus'], 
    margins=True,
    normalize='index'
) * 100

print("KPI 2: Leveransstatus per kasstyp (%)")
print(kpi2_leverans.round(1))
print()

In [ ]:
# === KPI 3: Kostpreferenser ===
kpi3_kost = df['kostpreferens'].value_counts().to_frame(name='Antal').reset_index()
kpi3_kost.columns = ['Kostpreferens', 'Antal']
kpi3_kost['Procent'] = (kpi3_kost['Antal'] / kpi3_kost['Antal'].sum() * 100).round(1)

print("KPI 3: Kostpreferenser")
print(kpi3_kost)
print()

In [ ]:
# === KPI 4: Genomsnittligt omdöme ===
kpi4_omdome = df.groupby('kasstyp')['omdömesbetyg'].agg([
    ('Genomsnittligt Betyg', 'mean'),
    ('Antal Omdömen', 'count'),
    ('Standardavvikelse', 'std')
]).round(2)

print("KPI 4: Omdömen per kasstyp")
print(kpi4_omdome)
print()

## METOD 2: Feature Engineering (Skapa nya variabler)

Skapa nya kolumner genom beräkningar för mer avancerad analys.

In [ ]:
# === Feature 1: Prenumerationslängd (i dagar) ===
df['pren_langd_dagar'] = (df['pren_avslutsdatum'] - df['pren_startdatum']).dt.days

kpi5_langd = df.groupby('kasstyp')['pren_langd_dagar'].agg([
    ('Genomsnittlig längd (dagar)', 'mean'),
    ('Median längd (dagar)', 'median'),
    ('Max längd (dagar)', 'max')
]).round(0)

print("KPI 5: Prenumerationslängd per kasstyp")
print(kpi5_langd)
print()

In [ ]:
# === Feature 2: Pauslängd (i dagar) ===
# Beräkna pauslängd för de som faktiskt pausade
df['paus_langd_dagar'] = (df['paus_till_filled'] - df['paus_från_filled']).dt.days

# För de som inte pausade, sätt till 0
df.loc[df['paus_från_filled'] == pd.Timestamp('9999-12-31'), 'paus_langd_dagar'] = 0

kpi6_pauslangd = df[df['paus_langd_dagar'] > 0]['paus_langd_dagar'].describe().round(1)

print("KPI 6: Pauslängd för prenumeranter som pausade")
print(kpi6_pauslangd)
print(f"Procent som pausade: {(df['paus_langd_dagar'] > 0).sum() / len(df) * 100:.1f}%")
print()

In [ ]:
# === Feature 3: Total intäkt per prenumeration ===
# Antag att veckapris multiplicerat med prenumerationslängd i veckor
df['total_intakt'] = df['veckapris'] * (df['pren_langd_dagar'] / 7)

kpi7_intakt = df.groupby('kasstyp')['total_intakt'].agg([
    ('Genomsnittlig Total Intäkt (SEK)', 'mean'),
    ('Total Intäkt per Kasstyp (SEK)', 'sum'),
    ('Antal Prenumerationer', 'count')
]).round(2)

print("KPI 7: Total intäkt per kasstyp")
print(kpi7_intakt)
print()

## METOD 3: Advanced Calculations (Komplexa beräkningar)

Skapa mer avancerade KPIer genom kombinationer av features.

In [ ]:
# === KPI 8: Churn Rate (Andel som slutade sina prenumerationer) ===
# Vi antar att alla i datasetet slutade (pren_avslutsdatum finns)
# Så churn = 100%, men vi kan se det per kasstyp och kostpreferens

churn_per_kass = df.groupby('kasstyp').size().reset_index(name='Antal')
churn_per_kass['Churn Rate (%)'] = 100.0  # Alla slutade i denna dataset

print("KPI 8: Churn Rate per kasstyp")
print(churn_per_kass)
print()
print("(Notering: I denna dataset är churn = 100% eftersom vi bara har slutade prenumerationer)")
print()

In [ ]:
# === KPI 9: Leveransreliabilitet (Success Rate) ===
# Andel "Levererad" av totala leveranser

leverans_reliability = df.groupby('kasstyp')['leveransstatus'].apply(
    lambda x: (x == 'Levererad').sum() / len(x) * 100
).reset_index()
leverans_reliability.columns = ['Kasstyp', 'Leverans Reliability (%)']
leverans_reliability['Leverans Reliability (%)'] = leverans_reliability['Leverans Reliability (%)'].round(1)

print("KPI 9: Leveransreliabilitet per kasstyp")
print(leverans_reliability)
print()

In [ ]:
# === KPI 10: Customer Lifetime Value (CLV) ===
# Intäkt per prenumeration justerad för längd

df['clv'] = df['veckapris'] * (df['pren_langd_dagar'] / 7)

clv_kost = df.groupby('kostpreferens')['clv'].agg([
    ('Genomsnittlig CLV (SEK)', 'mean'),
    ('Total CLV (SEK)', 'sum'),
    ('Antal Kunder', 'count')
]).round(2)

print("KPI 10: Customer Lifetime Value per kostpreferens")
print(clv_kost)
print()

## Visualiseringar

In [ ]:
# VIZ 1: Genomsnittligt veckapris per kasstyp
fig, ax = plt.subplots(figsize=(10, 6))
kpi1_pris['Genomsnittlig Pris (SEK)'].sort_values(ascending=False).plot(
    kind='bar', ax=ax, color='steelblue', edgecolor='black'
)
ax.set_title('Genomsnittligt Veckapris per Kasstyp', fontsize=14, fontweight='bold')
ax.set_ylabel('Pris (SEK)', fontsize=12)
ax.set_xlabel('Kasstyp', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('01_pris_per_kasstyp.png', dpi=300, bbox_inches='tight')
print("✓ Sparad: 01_pris_per_kasstyp.png")
plt.show()

In [ ]:
# VIZ 2: Leveransstatus per kasstyp
fig, ax = plt.subplots(figsize=(10, 6))
kpi2_leverans.drop('All', level=0, errors='ignore').iloc[:-1].plot(
    kind='bar', stacked=True, ax=ax, color=['#2ecc71', '#e74c3c']
)
ax.set_title('Leveransstatus per Kasstyp (%)', fontsize=14, fontweight='bold')
ax.set_ylabel('Andel (%)', fontsize=12)
ax.set_xlabel('Kasstyp', fontsize=12)
ax.legend(title='Status', loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('02_leveransstatus_per_kasstyp.png', dpi=300, bbox_inches='tight')
print("✓ Sparad: 02_leveransstatus_per_kasstyp.png")
plt.show()

In [ ]:
# VIZ 3: Kostpreferenser (Pie chart)
fig, ax = plt.subplots(figsize=(10, 8))
colors = sns.color_palette('Set2', len(kpi3_kost))
ax.pie(kpi3_kost['Antal'], labels=kpi3_kost['Kostpreferens'], autopct='%1.1f%%',
       colors=colors, startangle=90, textprops={'fontsize': 11})
ax.set_title('Distribution av Kostpreferenser', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('03_kostpreferenser_pie.png', dpi=300, bbox_inches='tight')
print("✓ Sparad: 03_kostpreferenser_pie.png")
plt.show()

In [ ]:
# VIZ 4: Omdömen per kasstyp
fig, ax = plt.subplots(figsize=(10, 6))
kpi4_omdome['Genomsnittligt Betyg'].sort_values(ascending=False).plot(
    kind='barh', ax=ax, color='coral', edgecolor='black'
)
ax.set_title('Genomsnittligt Omdöme per Kasstyp', fontsize=14, fontweight='bold')
ax.set_xlabel('Betyg (1-5)', fontsize=12)
ax.set_xlim(0, 5)
plt.tight_layout()
plt.savefig('04_omdome_per_kasstyp.png', dpi=300, bbox_inches='tight')
print("✓ Sparad: 04_omdome_per_kasstyp.png")
plt.show()

In [ ]:
# VIZ 5: Prenumerationslängd per kasstyp
fig, ax = plt.subplots(figsize=(10, 6))
kpi5_langd['Genomsnittlig längd (dagar)'].sort_values(ascending=False).plot(
    kind='bar', ax=ax, color='lightgreen', edgecolor='black'
)
ax.set_title('Genomsnittlig Prenumerationslängd per Kasstyp', fontsize=14, fontweight='bold')
ax.set_ylabel('Längd (dagar)', fontsize=12)
ax.set_xlabel('Kasstyp', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('05_pren_langd_per_kasstyp.png', dpi=300, bbox_inches='tight')
print("✓ Sparad: 05_pren_langd_per_kasstyp.png")
plt.show()

In [ ]:
# VIZ 6: Total intäkt per kasstyp
fig, ax = plt.subplots(figsize=(10, 6))
kpi7_intakt['Total Intäkt per Kasstyp (SEK)'].sort_values(ascending=False).plot(
    kind='bar', ax=ax, color='skyblue', edgecolor='black'
)
ax.set_title('Total Intäkt per Kasstyp', fontsize=14, fontweight='bold')
ax.set_ylabel('Intäkt (SEK)', fontsize=12)
ax.set_xlabel('Kasstyp', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('06_total_intakt.png', dpi=300, bbox_inches='tight')
print("✓ Sparad: 06_total_intakt.png")
plt.show()

In [ ]:
# VIZ 7: Leveransreliabilitet
fig, ax = plt.subplots(figsize=(10, 6))
leverans_reliability_sorted = leverans_reliability.sort_values('Leverans Reliability (%)', ascending=False)
ax.barh(leverans_reliability_sorted['Kasstyp'], leverans_reliability_sorted['Leverans Reliability (%)'],
        color='mediumseagreen', edgecolor='black')
ax.set_title('Leveransreliabilitet per Kasstyp', fontsize=14, fontweight='bold')
ax.set_xlabel('Reliabilitet (%)', fontsize=12)
ax.set_xlim(0, 100)
for i, v in enumerate(leverans_reliability_sorted['Leverans Reliability (%)']):
    ax.text(v + 1, i, f'{v:.1f}%', va='center')
plt.tight_layout()
plt.savefig('07_leveransreliabilitet.png', dpi=300, bbox_inches='tight')
print("✓ Sparad: 07_leveransreliabilitet.png")
plt.show()

In [ ]:
# VIZ 8: CLV per kostpreferens
fig, ax = plt.subplots(figsize=(10, 6))
clv_sorted = clv_kost['Genomsnittlig CLV (SEK)'].sort_values(ascending=False)
clv_sorted.plot(kind='bar', ax=ax, color='orchid', edgecolor='black')
ax.set_title('Customer Lifetime Value per Kostpreferens', fontsize=14, fontweight='bold')
ax.set_ylabel('CLV (SEK)', fontsize=12)
ax.set_xlabel('Kostpreferens', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('08_clv_per_kost.png', dpi=300, bbox_inches='tight')
print("✓ Sparad: 08_clv_per_kost.png")
plt.show()

In [ ]:
# VIZ 9: Distribution av antal portioner
fig, ax = plt.subplots(figsize=(10, 6))
df['antal_portioner'].hist(bins=30, ax=ax, color='teal', edgecolor='black')
ax.set_title('Distribution av Antal Portioner per Vecka', fontsize=14, fontweight='bold')
ax.set_xlabel('Antal Portioner', fontsize=12)
ax.set_ylabel('Frekvens', fontsize=12)
plt.tight_layout()
plt.savefig('09_antal_portioner_dist.png', dpi=300, bbox_inches='tight')
print("✓ Sparad: 09_antal_portioner_dist.png")
plt.show()

In [ ]:
# VIZ 10: Scatter plot - Veckapris vs Prenumerationslängd
fig, ax = plt.subplots(figsize=(10, 6))
scatter = ax.scatter(df['veckapris'], df['pren_langd_dagar'], 
                     c=df['antal_portioner'], cmap='viridis', s=50, alpha=0.6, edgecolor='black')
ax.set_title('Veckapris vs Prenumerationslängd', fontsize=14, fontweight='bold')
ax.set_xlabel('Veckapris (SEK)', fontsize=12)
ax.set_ylabel('Prenumerationslängd (dagar)', fontsize=12)
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Antal Portioner', fontsize=11)
plt.tight_layout()
plt.savefig('10_pris_vs_langd.png', dpi=300, bbox_inches='tight')
print("✓ Sparad: 10_pris_vs_langd.png")
plt.show()

## Sammanfattning av alla KPIer

In [ ]:
print("\n" + "="*70)
print("SAMMANFATTNING AV ALLA KPIer")
print("="*70)

print("\n📊 METOD 1: Simple Aggregation")
print("  KPI 1 - Pris-analys per kasstyp")
print("  KPI 2 - Leveransstatus per kasstyp")
print("  KPI 3 - Kostpreferenser")
print("  KPI 4 - Omdömen per kasstyp")

print("\n🔧 METOD 2: Feature Engineering")
print("  KPI 5 - Prenumerationslängd per kasstyp")
print("  KPI 6 - Pauslängd för prenumeranter")
print("  KPI 7 - Total intäkt per kasstyp")

print("\n🎯 METOD 3: Advanced Calculations")
print("  KPI 8 - Churn Rate per kasstyp")
print("  KPI 9 - Leveransreliabilitet")
print("  KPI 10 - Customer Lifetime Value per kostpreferens")

print("\n📁 Visualiseringar sparade:")
print("  01_pris_per_kasstyp.png")
print("  02_leveransstatus_per_kasstyp.png")
print("  03_kostpreferenser_pie.png")
print("  04_omdome_per_kasstyp.png")
print("  05_pren_langd_per_kasstyp.png")
print("  06_total_intakt.png")
print("  07_leveransreliabilitet.png")
print("  08_clv_per_kost.png")
print("  09_antal_portioner_dist.png")
print("  10_pris_vs_langd.png")

print("\n✓ Alla KPIer och visualiseringar klar!")
print("="*70)